In [1]:
pip install sklearn-crfsuite

In [2]:
whole = []

with open("NE.txt") as txt:
  for line in txt:
    whole.append(line)
  

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics





In [4]:
 a = open(r'location.csv', encoding="utf8")
LOCATIONS = a.read().splitlines()
for i in range(len(LOCATIONS)):
  LOCATIONS[i] = LOCATIONS[i].lower()

for location in LOCATIONS:
  if (location.find('�') != -1):
    LOCATIONS.remove(location)

a.close()

In [5]:
morphanalysis = []
with open("NE.ma.txt") as txt:
  morphanalysis = txt.readlines()

morphs = []
X_train = []
for morph in morphanalysis:
  morph = morph.split()
  morphs.append(morph)

import itertools
key_f = lambda x: x[0]

morphlist = []
for key, group in itertools.groupby(morphs, key_f):
  morphlist.append(list(group))


for morph in morphlist:
  sentmorph = []
  featurelist = []


  for m in morph:
    analysis = m[2].split('+')
    #STEM
    stem = analysis[0]
    
    #POS
    if stem == '*UNKNOWN*':
      POS = 0
    else:
      POS = analysis[1]

    #PROP
    if 'Prop' in analysis:
      PROP = 1
    else:
      PROP = 0

    #NCS
    if analysis[-1] == 'Nom':
      NCS = 'Nom'
    
    elif analysis[-1] == 'Acc':
      NCS = 'Acc'
    
    elif analysis[-1] == 'Dat':
      NCS = 'Dat'

    elif analysis[-1] == 'Abl':
      NCS = 'Abl'
    
    elif analysis[-1] == 'Loc':
      NCS = 'Loc'
  
    elif analysis[-1] == 'Gen':
      NCS = 'Gen'

    elif analysis[-1] == 'Ins':
      NCS = 'Ins'

    elif analysis[-1] == 'Equ':
      NCS = 'Equ'
    
    else:    
      NCS = 0

    #SS
    if len(sentmorph) == 0:
      SS = 1
    else:
      SS = 0

    #OCS
    if m[1][0].isupper() == True:
      OCS = 1
    else:
      OCS = 0

    #LOCLEXICON
    if m[1].lower() in LOCATIONS:
      LOCLEXICON = 1
    else:
      LOCLEXICON = 0
   
    """   
    #INF
    if len(analysis) > 2:
      temp = analysis[2]
      for e in analysis[3:]:
        temp += '+' + e
      INF = temp
    else:
      INF = 0
    """
    
    feature = {
        #"INF": INF,
        "Stem": stem,
        "POS": POS,
        "PROP" : PROP,
        "NCS": NCS,
        "SS": SS,
        "OCS": OCS,
        "LOCLEXICON": LOCLEXICON,
    }
    sentmorph.append(feature)
  
  X_train.append(sentmorph)


X_train1 = []
X_train2 = []
X_train3 = []
X_train4 = []
X_train5 = []
i=1
for element in X_train:
    if i%5 == 1:
      X_train1.append(element)
    if i%5 == 2:
      X_train2.append(element)
    if i%5 == 3:
      X_train3.append(element)
    if i%5 == 4:
      X_train4.append(element)
    if i%5 == 0:
      X_train5.append(element)
    i += 1


print(X_train[6])


[{'Stem': 'Venedik', 'POS': 'Noun', 'PROP': 1, 'NCS': 'Nom', 'SS': 1, 'OCS': 1, 'LOCLEXICON': 0}, {'Stem': 'film', 'POS': 'Noun', 'PROP': 0, 'NCS': 'Nom', 'SS': 0, 'OCS': 1, 'LOCLEXICON': 0}, {'Stem': 'festival', 'POS': 'Noun', 'PROP': 0, 'NCS': 'Nom', 'SS': 0, 'OCS': 1, 'LOCLEXICON': 0}, {'Stem': ',', 'POS': 'Punct', 'PROP': 0, 'NCS': 0, 'SS': 0, 'OCS': 0, 'LOCLEXICON': 0}, {'Stem': 'büyük', 'POS': 'Adj', 'PROP': 0, 'NCS': 0, 'SS': 0, 'OCS': 0, 'LOCLEXICON': 0}, {'Stem': 'final', 'POS': 'Noun', 'PROP': 0, 'NCS': 'Dat', 'SS': 0, 'OCS': 0, 'LOCLEXICON': 0}, {'Stem': 'heyecan', 'POS': 'Noun', 'PROP': 0, 'NCS': 0, 'SS': 0, 'OCS': 0, 'LOCLEXICON': 0}, {'Stem': 'yaklaş', 'POS': 'Verb', 'PROP': 0, 'NCS': 0, 'SS': 0, 'OCS': 0, 'LOCLEXICON': 0}]


In [6]:
import re
from nltk.tokenize import word_tokenize


orglist =[]
loclist =[]
personlist =[]
labels = []
for e in whole:
  org = re.findall(r'(?<=<b_enamex TYPE=\"ORGANIZATION\">)[\w\s]+(?=<e_enamex>)', e)
  loc = re.findall(r'(?<=<b_enamex TYPE=\"LOCATION\">)[\w\s]+(?=<e_enamex>)', e)
  person = re.findall(r'(?<=<b_enamex TYPE=\"PERSON\">)[\w\s]+(?=<e_enamex>)', e)
  
  for o in org:
    orglist.append(o)
  for l in loc:
    loclist.append(l)
  for p in person:
    personlist.append(p)



In [7]:
full = []
for line in whole:
  line = line.replace('<b_enamex TYPE="ORGANIZATION">' ,'' )
  line = line.replace('<b_enamex TYPE="LOCATION">' ,'' )
  line = line.replace('<b_enamex TYPE="PERSON">' ,'' )
  line = line.replace('<e_enamex>' ,'' )
  line = line.replace('\n', '')
  line = line.replace(' \'','\'')
  full.append(line)



y_train = []

for sent in full:
  tokensent = str(sent).split()
  for loc in loclist:
      if loc in sent:
        loc = str(loc).split()
        for s in tokensent:
          if loc[0] == s:
            x = tokensent.index(s)
            tokensent[x] = tokensent[x].replace(s,'B-LOCATION')          
          else:
            for i in range(1,len(loc)):
              if(loc[i] == s):
                x = tokensent.index(s)
                tokensent[x] = tokensent[x].replace(s,'I-LOCATION')          
    
  for org in orglist:
      if org in sent:
        org = str(org).split()
        for s in tokensent:
          if org[0] == s:
            x = tokensent.index(s)
            tokensent[x] = tokensent[x].replace(s,'B-ORGANIZATION')          
          else:
            for i in range(1,len(org)):
              if(org[i] == s):
                x = tokensent.index(s)
                tokensent[x] = tokensent[x].replace(s,'I-ORGANIZATION')     

  for person in personlist:
      if person in sent:
        person = str(person).split()
        for s in tokensent:
          if person[0] == s:
            x = tokensent.index(s)
            tokensent[x] = tokensent[x].replace(s,'B-PERSON')          
          else:
            for i in range(1,len(person)):
              if(person[i] == s):
                x = tokensent.index(s)
                tokensent[x] = tokensent[x].replace(s,'I-PERSON')  
    
  for s in tokensent:
    if (((((s == 'B-LOCATION' or s == 'B-PERSON') or s == 'B-ORGANIZATION') or s == 'I-LOCATION') or s == 'I-PERSON') or s == 'I-ORGANIZATION'):
      ;
    else:
      x = tokensent.index(s)
      tokensent[x] = tokensent[x].replace(s,'O')


  y_train.append(tokensent)    


y_train1 = []
y_train2 = []
y_train3 = []
y_train4 = []
y_train5 = []
i=1
for element in y_train:
    if i%5 == 1:
      y_train1.append(element)
    if i%5 == 2:
      y_train2.append(element)
    if i%5 == 3:
      y_train3.append(element)
    if i%5 == 4:
      y_train4.append(element)
    if i%5 == 0:
      y_train5.append(element)
    i += 1


print(y_train[0])
 


['O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)


x_train1 = []
x_train2 = []
x_train3 = []
x_train4 = []
x_train5 = []

Y_train1 = []
Y_train2 = []
Y_train3 = []
Y_train4 = []
Y_train5 = []

count = 0

for i in range(len(X_train1)):
  if  len(X_train1[i]) == len(y_train1[i]):
    count = count +1 
    x_train1.append(X_train1[i])
    Y_train1.append(y_train1[i])
  
  else:
    count += 1  


for i in range(len(X_train2)):
  if  len(X_train2[i]) == len(y_train2[i]):
    x_train2.append(X_train2[i])
    Y_train2.append(y_train2[i])
  
  else:
    count += 1


for i in range(len(X_train3)):
  if  len(X_train3[i]) == len(y_train3[i]):
    x_train3.append(X_train3[i])
    Y_train3.append(y_train3[i])  

  else:
    count += 1


for i in range(len(X_train4)):
  if  len(X_train4[i]) == len(y_train4[i]):
    x_train4.append(X_train4[i])
    Y_train4.append(y_train4[i])

  else:
    count += 1


for i in range(len(X_train5)):
  if  len(X_train5[i]) == len(y_train5[i]):
    x_train5.append(X_train5[i])
    Y_train5.append(y_train5[i])
  
  else:
    count += 1

print(count)

2373


In [9]:
X_train_fold1 = x_train2 + x_train3 + x_train4 + x_train5
y_train_fold1 = Y_train2 + Y_train3 + Y_train4 + Y_train5
X_test_fold1 = x_train1
y_test_fold1 = Y_train1


X_train_fold2 = x_train1 + x_train3 + x_train4 + x_train5
y_train_fold2 = Y_train1 + Y_train3 + Y_train4 + Y_train5
X_test_fold2 = x_train2
y_test_fold2 = Y_train2


X_train_fold3 = x_train1 + x_train2 + x_train4 + x_train5
y_train_fold3 = Y_train1 + Y_train2 + Y_train4 + Y_train5
X_test_fold3 = x_train3
y_test_fold3 = Y_train3


X_train_fold4 = x_train1 + x_train2 + x_train3 + x_train5
y_train_fold4 = Y_train1 + Y_train2 + Y_train3 + Y_train5
X_test_fold4 = x_train4
y_test_fold4 = Y_train4


X_train_fold5 = x_train1 + x_train2 + x_train3 + x_train4
y_train_fold5 = Y_train1 + Y_train2 + Y_train3 + Y_train4
X_test_fold5 = x_train5
y_test_fold5 = Y_train5

In [10]:
crf1 = crf.fit(X_train_fold1, y_train_fold1)

In [11]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOCATION',
 'B-PERSON',
 'I-PERSON',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-LOCATION']

In [12]:
y_pred = crf1.predict(X_test_fold1)
f1_score_fold1 = metrics.flat_f1_score(y_test_fold1, y_pred, average='weighted', labels=labels)
print(f1_score_fold1)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(
    y_test_fold1, y_pred, labels=sorted_labels, digits=3
))

0.8603052792607571
                precision    recall  f1-score   support

    B-LOCATION      0.934     0.917     0.925       480
    I-LOCATION      0.500     0.286     0.364        28
B-ORGANIZATION      0.930     0.720     0.812       446
I-ORGANIZATION      0.768     0.684     0.724       228
      B-PERSON      0.908     0.888     0.898       890
      I-PERSON      0.853     0.883     0.868       283

     micro avg      0.894     0.834     0.863      2355
     macro avg      0.816     0.730     0.765      2355
  weighted avg      0.893     0.834     0.860      2355



In [13]:
crf2 = crf.fit(X_train_fold2, y_train_fold2)

In [14]:
y_pred = crf2.predict(X_test_fold2)
f1_score_fold2 = metrics.flat_f1_score(y_test_fold2, y_pred, average='weighted', labels=labels)
print(f1_score_fold2)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test_fold2, y_pred, labels=sorted_labels, digits=3
))

0.8622323890579712
                precision    recall  f1-score   support

    B-LOCATION      0.943     0.915     0.929       492
    I-LOCATION      0.500     0.550     0.524        20
B-ORGANIZATION      0.881     0.715     0.789       361
I-ORGANIZATION      0.689     0.737     0.712       198
      B-PERSON      0.911     0.880     0.895       937
      I-PERSON      0.831     0.890     0.860       292

     micro avg      0.877     0.848     0.862      2300
     macro avg      0.792     0.781     0.785      2300
  weighted avg      0.880     0.848     0.862      2300



In [15]:
crf3 = crf.fit(X_train_fold3, y_train_fold3)

In [16]:
y_pred = crf3.predict(X_test_fold3)
f1_score_fold3 = metrics.flat_f1_score(y_test_fold3, y_pred, average='weighted', labels=labels)
print(f1_score_fold3)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test_fold3, y_pred, labels=sorted_labels, digits=3
))

0.8573464442121052
                precision    recall  f1-score   support

    B-LOCATION      0.919     0.926     0.922       488
    I-LOCATION      0.565     0.433     0.491        30
B-ORGANIZATION      0.896     0.765     0.825       370
I-ORGANIZATION      0.714     0.725     0.719       182
      B-PERSON      0.892     0.871     0.882       933
      I-PERSON      0.858     0.816     0.836       310

     micro avg      0.876     0.841     0.858      2313
     macro avg      0.807     0.756     0.779      2313
  weighted avg      0.875     0.841     0.857      2313



In [17]:
crf4 = crf.fit(X_train_fold4, y_train_fold4)

In [18]:
y_pred = crf.predict(X_test_fold4)
f1_score_fold4 = metrics.flat_f1_score(y_test_fold4, y_pred, average='weighted', labels=labels)
print(f1_score_fold4)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test_fold4, y_pred, labels=sorted_labels, digits=3
))

0.8519396672433779
                precision    recall  f1-score   support

    B-LOCATION      0.952     0.885     0.917       495
    I-LOCATION      0.880     0.478     0.620        46
B-ORGANIZATION      0.914     0.752     0.825       423
I-ORGANIZATION      0.794     0.733     0.762       210
      B-PERSON      0.885     0.870     0.878       933
      I-PERSON      0.733     0.862     0.792       261

     micro avg      0.875     0.832     0.853      2368
     macro avg      0.860     0.763     0.799      2368
  weighted avg      0.879     0.832     0.852      2368



In [19]:
crf5 = crf.fit(X_train_fold5, y_train_fold5)

In [20]:
y_pred = crf.predict(X_test_fold5)
f1_score_fold5 = metrics.flat_f1_score(y_test_fold5, y_pred, average='weighted', labels=labels)
print(f1_score_fold5)


sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test_fold5, y_pred, labels=sorted_labels, digits=3
))

0.847699692593214
                precision    recall  f1-score   support

    B-LOCATION      0.926     0.882     0.903       524
    I-LOCATION      0.375     0.243     0.295        37
B-ORGANIZATION      0.912     0.766     0.833       367
I-ORGANIZATION      0.674     0.674     0.674       178
      B-PERSON      0.908     0.851     0.879       968
      I-PERSON      0.808     0.874     0.839       293

     micro avg      0.874     0.825     0.849      2367
     macro avg      0.767     0.715     0.737      2367
  weighted avg      0.874     0.825     0.848      2367



In [21]:
avg_f1_score = (f1_score_fold1 + f1_score_fold2 + f1_score_fold3 + f1_score_fold4 + f1_score_fold5)/5
print(avg_f1_score)

0.855904694473485
